<a href="https://colab.research.google.com/github/NumiKun/Dicoding_BPML/blob/main/Generative_AI_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cek Penggunaan GPU

In [1]:
# Menggunakan local environment
# nvidia-smi
# Menggunakan Google Colab
!nvidia-smi

Sat Apr 19 10:51:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Instalasi Libraries

In [2]:
!pip uninstall -y jax jaxlib

Found existing installation: jax 0.6.0
Uninstalling jax-0.6.0:
  Successfully uninstalled jax-0.6.0
Found existing installation: jaxlib 0.6.0
Uninstalling jaxlib-0.6.0:
  Successfully uninstalled jaxlib-0.6.0


In [3]:
!pip install "jax[cuda]>=0.4.13" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached jax-0.6.0-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.6.0-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached nvidia_cudnn_cu12-9.8.0.87-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
Using cached jax-0.6.0-py3-none-any.whl (2.3 MB)
Using cached jaxlib-0.6.0-cp311-cp311-manylinux2014_x86_64.whl (87.8 MB)
Using cached nvidia_cudnn_cu12-9.8.0.87-py3-none-manylinux_2_27_x86_64.whl (698.0 MB)
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.1.0.70
    Uninstalling nvidia-cudnn-cu12-9.1.0.70:
      Successfully uninstalled nvidia-cudnn-cu12-9.1.0.70
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and pla

In [4]:
!pip install diffusers==0.11.1
!pip install -q accelerate transformers bitsandbytes==0.35.0 safetensors xformers

In [5]:
!pip uninstall -y huggingface_hub
!pip install --upgrade huggingface_hub

Found existing installation: huggingface-hub 0.30.2
Uninstalling huggingface-hub-0.30.2:
  Successfully uninstalled huggingface-hub-0.30.2
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)


In [6]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import matplotlib.pyplot as plt

ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py)

### Mengunduh Dataset

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
# Jika menggunakan local env kecepatan download pre-trained model tergantung koneksi internet Anda.

### Menentukan CUDA sebagai Komponen Utama

In [ ]:
pipe = pipe.to('cuda')

### mengoptimalkan penggunaan memori dan meningkatkan efisiensi komputasi

In [ ]:
pipe.enable_attention_slicing()
pipe.enable_xformers_memory_efficient_attention()

### Generate Gambar

#### Prompt

In [ ]:
prompt = 'a dog using combat machete to war, full HD'
img = pipe(prompt).images[0]

#### Seed

In [ ]:
seed = 2000generator = torch.Generator('cuda').manual_seed(seed)
img = pipe(prompt, generator=generator).images[0]
img

#### Guidance Scale

In [ ]:
plt.figure(figsize=(18,8))for i in range(1, 6):

  n_guidance = i + 3
  generator = torch.Generator("cuda").manual_seed(seed)
  img = pipe(prompt, guidance_scale=n_guidance, generator=generator).images[0]

  plt.subplot(1,5,i)
  plt.title('guidance_scale: {}'.format(n_guidance))
  plt.imshow(img)
  plt.axis('off')

plt.show()

#### Inference Step

In [ ]:
prompt = "A man using baby shark costume at the moon"
generator = torch.Generator("cuda")
img = pipe(prompt, num_inference_steps=999, generator=generator).images[0]
img

Melihat Perbedaan per Generasi

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(18,8))
for i in range(1, 10):
  n_steps = i * 1
  #print(n_steps)
  generator = torch.Generator('cuda')
  img = pipe(prompt, num_inference_steps=n_steps, generator=generator).images[0]

  plt.subplot(1, 10, i)
  plt.title('num_inference_steps: {}'.format(n_steps))
  plt.imshow(img)
  plt.axis('off')
plt.show()

#### Negative Prompt

In [ ]:
num_images = 5
prompt = 'a man enjoying sunset at bali beach with his girlfriend'
neg_prompt = 'coconut, knife, straw, building'


imgs = pipe(prompt, negative_prompt = neg_prompt, num_images_per_prompt=num_images).images
grid = grid_img(imgs, rows = 1, cols = 5, scale=0.75)
grid